## Budget optimizer dashboard

In [2]:
# 
import pandas as pd
import sys
import json
import ipywidgets as widgets
encoding = sys.getdefaultencoding()

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

Basic settings:
* period: month, week, day
* Margin size: 1σ or 2σ
* Margin: upper, lower or zero boundary (optimistic, realistic  pessimistic)
* List of components we will consider for budget optimization (channel, platform and so on) 


Advanced settings:
* Filter: type, size, period (for outliers filtering)
* Error threshold in model? 


In [6]:
granularity = 'week'
margin_size = 1
margin_bound = 'upper' # 'mean' 'lower'

filter_type = 'avrg'   # 'median'
filter_length = 10     # number of points in sequence 
error_threshhold = 0   # I'm going to define it after optimisation task
zero_cross = True

independent_var = 'marketingInvestment'
dependent_var   = 'netRevenue'

In [ ]:
if zero_cross:
    reggre_coeff_set = ['a','b']
else:
    reggre_coeff_set = ['a','b','c']